<a href="https://colab.research.google.com/github/hufsaim/T10402201/blob/master/ch03/linear_reg_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://d2l.ai/chapter_linear-networks/linear-regression-scratch.html

In [ ]:
%matplotlib inline
import random
import torch

- define functions

In [ ]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i +
                                                   batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

def linreg(X, w, b):  #@save
    """The linear regression model."""
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

def sgd(params, lr, batch_size):  #@save
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

- load examples

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print('features:', features[0], '\nlabel:', labels[0])
print('features:', features[1], '\nlabel:', labels[1])
print('features:', features[2], '\nlabel:', labels[2])
print('features:', features[-1], '\nlabel:', labels[-1])

features: tensor([0.5365, 0.6737]) 
label: tensor([2.9790])
features: tensor([-0.9289, -1.9422]) 
label: tensor([8.9474])
features: tensor([0.8484, 0.1188]) 
label: tensor([5.4913])
features: tensor([2.3479, 0.4363]) 
label: tensor([7.4335])


- Parameters (weights, bias)

In [ ]:
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

- Hyperparameters

In [ ]:
batch_size = 10
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

- Train a model

In [ ]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.029268
epoch 2, loss 0.000104
epoch 3, loss 0.000048


- Check results

In [ ]:
print(w[0],w[1],b)

tensor([2.0003], grad_fn=<SelectBackward>) tensor([-3.3994], grad_fn=<SelectBackward>) tensor([4.1992], requires_grad=True)
